In [23]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import os
import logging

# 로깅 설정
log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)

log_file_path = os.path.join(log_folder, 'weather_data_fetch.log')
logging.basicConfig(filename=log_file_path, format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

class WeatherDataFetcher:
    def __init__(self, api_key_file):
        self.api_key = self.read_api_key(api_key_file)
        if not self.api_key:
            logging.error("API key reading failed. Exiting.")
            exit(1)
        self.api_url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    
    @staticmethod
    def read_api_key(file_path):
        try:
            with open(file_path, 'r') as f:
                return f.read().strip()
        except FileNotFoundError:
            logging.error("API key file not found.")
            return None

    def fetch_all_weather_data(self, params):
        # 요청 URL을 생성합니다.
        request_url = requests.Request('GET', self.api_url, params=params).prepare().url
        print(f"Request URL: {request_url}")  # 생성된 URL을 출력합니다.

        try:
            res = requests.get(self.api_url, params=params)
            res.raise_for_status()
        except requests.RequestException as e:
            logging.error(f"API request failed for params {params}: {e}")
            return None
    
        root = ET.fromstring(res.text)
        item_list = root.findall('./body/items/item')
        if not item_list:
            logging.warning(f"No items found in the XML response for params {params}.")
            return None

        result_list = []
        for item in item_list:
            item_dict = {}
            for elem in item:
                tag = elem.tag
                value = elem.text or "Nan"
                item_dict[tag] = value
            result_list.append(item_dict)

        return result_list

    def fetch_data_for_location_and_time(self, base_date, base_time, nx, ny, custom_file_name=None, num_rows=809, page_range=1):
        params = {
            'serviceKey': self.api_key,
            'pageNo': 1,
            'numOfRows': num_rows,
            'dataType': 'XML',
            'base_date': base_date,
            'base_time': base_time,
            'nx': nx,
            'ny': ny
        }

        result_dicts = []
        for i in range(1, page_range + 1):
            params['pageNo'] = i
            result_list = self.fetch_all_weather_data(params)
            if result_list:
                result_dicts.extend(result_list)

        df = pd.DataFrame(result_dicts)
        if df.empty:
            logging.warning(f"No data to save for params {params}.")
            return None

        grouped = df.groupby(['baseDate', 'baseTime', 'fcstDate', 'fcstTime', 'nx', 'ny'])
        new_df = pd.DataFrame()

        for name, group in grouped:
            record = {
                'baseDate': name[0],
                'baseTime': name[1],
                'fcstDate': name[2],
                'fcstTime': name[3],
                'nx': name[4],
                'ny': name[5]
            }
            for i, row in group.iterrows():
                category = row['category']
                value = row['fcstValue']
                record[category] = value

            new_df = pd.concat([new_df, pd.DataFrame([record])], ignore_index=True)
        
        data_folder = 'data'
        if not os.path.exists(data_folder):
            os.makedirs(data_folder)
            
        if custom_file_name:
            csv_file_path = os.path.join(data_folder, custom_file_name)
        else:
            csv_file_name = f'weather_data_{base_date}_{base_time}_{nx}_{ny}.csv'
            csv_file_path = os.path.join(data_folder, csv_file_name)
        new_df.to_csv(csv_file_path, index=False, encoding='cp949')
        logging.info(f"Data has been saved to {csv_file_path}")
        return new_df

In [26]:
import os
import pandas as pd
import datetime
#from WeatherDataFetcher import WeatherDataFetcher  # WeatherDataFetcher 클래스를 정의한 모듈을 불러옵니다.

# API 키 파일과 날짜, 시간 설정
api_key_file = "KMA_API_KEY(Decoding).txt"
base_date = datetime.datetime.now().strftime('%Y%m%d')  # 시스템의 현재 날짜를 YYYYMMDD 형식으로 가져옵니다.
base_time = '0500'  # 고정 시간

# WeatherDataFetcher 인스턴스 생성
fetcher = WeatherDataFetcher(api_key_file)

# 격자 좌표 파일 불러오기
grid_points_path = 'API_grid_points.csv'  # 실제 파일 경로로 변경해야 합니다.
grid_points_df = pd.read_csv(grid_points_path, encoding='cp949')

# 각 격자 좌표에 대해 API 호출
for index, row in grid_points_df.iterrows():
    nx = row['nx']
    ny = row['ny']
    fetcher.fetch_data_for_location_and_time(base_date, base_time, nx, ny)

Request URL: https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=SGD3J7nUrX%2Bod%2B6SN3Fu%2BZB4N9DS13PXL4NV7jwYaFGTFjJSmN4Uk2eIr3ErirViBOIzt2L%2FiYtsg%2BItaPW6YA%3D%3D&pageNo=1&numOfRows=809&dataType=XML&base_date=20240221&base_time=0500&nx=52&ny=79
Request URL: https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=SGD3J7nUrX%2Bod%2B6SN3Fu%2BZB4N9DS13PXL4NV7jwYaFGTFjJSmN4Uk2eIr3ErirViBOIzt2L%2FiYtsg%2BItaPW6YA%3D%3D&pageNo=1&numOfRows=809&dataType=XML&base_date=20240221&base_time=0500&nx=59&ny=74


In [28]:
import pandas as pd
import os

# 데이터 폴더 내의 모든 파일을 순회하며 분석
data_folder = './data'
for file_name in os.listdir(data_folder):
    if file_name.endswith('.csv'):
        weather_data_path = os.path.join(data_folder, file_name)
        weather_df = pd.read_csv(weather_data_path, encoding='cp949')

        # nx, ny 좌표 추출
        nx, ny = file_name.split('_')[-2], file_name.split('_')[-1].replace('.csv', '')

        # 체감온도 계산 함수
        def calculate_heat_index(temp, humidity):
            return temp + (humidity / 10)

        # 폭염과 강풍 분석 함수
        def analyze_weather_conditions(df):
            heat_indexes = df.apply(lambda x: calculate_heat_index(x['TMX'], x['REH']), axis=1)
            heat_wave = heat_indexes.max() >= 33
            strong_wind = df['WSD'].max() > 50.4
            return heat_wave, strong_wind

        # 날짜별 분석 함수
        def analyze_daily_weather(df, base_date):
            daily_report = ""
            for date in sorted(df['fcstDate'].unique()):
                if date > base_date + 2:
                    continue
                daily_df = df[df['fcstDate'] == date]
                max_temp = daily_df['TMX'].max()
                min_temp = daily_df['TMN'].min() if not pd.isna(daily_df['TMN'].min()) else daily_df['TMP'].min()
                max_precipitation = daily_df['PCP'].max()
                max_snowfall = daily_df['SNO'].max()

                daily_report += f"날짜: {date}({nx}, {ny})\n"
                daily_report += f"최고 기온: {max_temp}°C\n"
                daily_report += f"최저 기온: {min_temp}°C\n"
                daily_report += f"최대 강수량: {max_precipitation}\n"
                daily_report += f"최대 적설량: {max_snowfall}\n\n"
            return daily_report

        # 기준 날짜 설정
        base_date = min(weather_df['fcstDate'].unique())

        # 폭염과 강풍 분석
        heat_wave, strong_wind = analyze_weather_conditions(weather_df)

        # 날짜별 날씨 분석
        daily_weather_report = analyze_daily_weather(weather_df, base_date)

        # 최종 보고서 출력
        report = f"오늘의 날씨 분석 보고서({nx}, {ny}):\n"
        report += f"폭염 여부: {'있음' if heat_wave else '없음'}\n"
        report += f"강풍 여부: {'있음' if strong_wind else '없음'}\n\n"
        report += daily_weather_report
        report += "※ 참고사항: 보고서 내용은 기상청 예보 발표 시간 5:00am 이후 수치에 기반하여 작성되었습니다."

        # 보고서를 파일로 저장
        report_file_name = f'weather_report_{nx}_{ny}.txt'
        report_file_path = os.path.join(data_folder, report_file_name)
        with open(report_file_path, 'w', encoding='utf-8') as report_file:
            report_file.write(report)

In [39]:
import pandas as pd
import os

# 지역명 매핑을 위한 데이터를 로드합니다.
region_mapping_path = './region_grid_data_sigungu.csv'  # 실제 파일 경로로 수정해야 합니다.
region_mapping_df = pd.read_csv(region_mapping_path, encoding='cp949')

# 지역명 매핑을 위한 딕셔너리 생성
region_dict = {f"{row['nx']}_{row['ny']}": row['지역명'] for _, row in region_mapping_df.iterrows()}

# 데이터 폴더 내의 모든 파일을 순회하며 분석
data_folder = './data'  # 실제 데이터 폴더 경로로 수정해야 합니다.
for file_name in os.listdir(data_folder):
    if file_name.endswith('.csv'):
        # nx, ny 좌표 추출 및 지역명 매핑
        nx, ny = file_name.split('_')[-2], file_name.split('_')[-1].replace('.csv', '')
        region_name = region_dict.get(f"{nx}_{ny}", "알 수 없는 지역")

        weather_data_path = os.path.join(data_folder, file_name)
        weather_df = pd.read_csv(weather_data_path, encoding='cp949')


        # 체감온도 계산 함수
        def calculate_heat_index(temp, humidity):
            return temp + (humidity / 10)

        # 폭염과 강풍 분석 함수
        def analyze_weather_conditions(df):
            heat_indexes = df.apply(lambda x: calculate_heat_index(x['TMX'], x['REH']), axis=1)
            heat_wave = heat_indexes.max() >= 33
            strong_wind = df['WSD'].max() > 50.4
            return heat_wave, strong_wind

        # 날짜별 분석 함수
        def analyze_daily_weather(df, base_date):
            daily_report = ""
            for date in sorted(df['fcstDate'].unique()):
                if date > base_date + 2:
                    continue
                daily_df = df[df['fcstDate'] == date]
                print(daily_df)
                max_temp = daily_df['TMX'].max()
                min_temp = daily_df['TMN'].min() if not pd.isna(daily_df['TMN'].min()) else daily_df['TMP'].min()
                max_precipitation = daily_df['PCP'].max()
                max_snowfall = daily_df['SNO'].max()

                daily_report += f"날짜: {date}\n"
                daily_report += f"최고 기온: {max_temp}°C\n"
                daily_report += f"최저 기온: {min_temp}°C\n"
                daily_report += f"최대 강수량: {max_precipitation}\n"
                daily_report += f"최대 적설량: {max_snowfall}\n\n"
            return daily_report
        
        # 최종 보고서 출력
        report = f"오늘의 날씨 분석 보고서:\n"
        report += f"폭염 여부: {'있음' if heat_wave else '없음'}\n"
        report += f"강풍 여부: {'있음' if strong_wind else '없음'}\n\n"
        report += daily_weather_report
        report += "※ 참고사항: 보고서 내용은 기상청 예보 발표 시간 5:00am 이후 수치에 기반하여 작성되었습니다."

        # 보고서를 파일로 저장
        report_file_name = f'weather_report_{region_name}.txt'
        report_file_path = os.path.join(data_folder+"/report", report_file_name)
        with open(report_file_path, 'w', encoding='utf-8') as report_file:
            report_file.write(report)